# Some simple demos of approaches to Bayesian inference with unknown errors

In [8]:
import numpy as np
from scipy.stats import invgamma, norm

## Treat the error as a parameter

In [9]:
def log_posterior(y, x, m, b, sigma2, m_prior, b_prior, sigma2_prior):
    """
    Calculate the log posterior given parameters.
    """
    # Log likelihood
    residuals = y - (m * x + b)
    log_likelihood = -0.5 * np.sum(residuals**2 / sigma2) - (len(y) / 2) * np.log(
        2 * np.pi * sigma2
    )

    # Log priors
    log_prior_m = norm.logpdf(m, loc=m_prior[0], scale=np.sqrt(m_prior[1]))
    log_prior_b = norm.logpdf(b, loc=b_prior[0], scale=np.sqrt(b_prior[1]))
    log_prior_sigma2 = invgamma.logpdf(sigma2, a=sigma2_prior[0], scale=sigma2_prior[1])

    return log_likelihood + log_prior_m + log_prior_b + log_prior_sigma2

In [10]:
def metropolis_hastings(
    y,
    x,
    n_iterations=1000,
    burn_in=200,
    m_prior=(0, 10),
    b_prior=(0, 10),
    sigma2_prior=(1, 1),
    proposal_std=(0.1, 0.1, 0.1),
):
    # Proposal distribution std deviation
    prop_m, prop_b, prop_log_sigma2 = proposal_std

    # Initialization
    m_current = np.random.normal(m_prior[0], np.sqrt(m_prior[1]))
    b_current = np.random.normal(b_prior[0], np.sqrt(b_prior[1]))
    sigma2_current = invgamma.rvs(a=sigma2_prior[0], scale=sigma2_prior[1])

    # Storage for samples
    m_samples = []
    b_samples = []
    sigma2_samples = []

    # Current log posterior
    current_log_post = log_posterior(
        y, x, m_current, b_current, sigma2_current, m_prior, b_prior, sigma2_prior
    )

    accepted = 0
    for iteration in range(n_iterations):
        # Propose new m and b
        m_proposal = m_current + np.random.normal(0, prop_m)
        b_proposal = b_current + np.random.normal(0, prop_b)
        # Propose new sigma^2 using log proposal (ensures positivity)
        log_sigma2_proposal = np.log(sigma2_current) + np.random.normal(
            0, prop_log_sigma2
        )
        sigma2_proposal = np.exp(log_sigma2_proposal)

        # Calculate log posterior for proposed values
        proposed_log_post = log_posterior(
            y,
            x,
            m_proposal,
            b_proposal,
            sigma2_proposal,
            m_prior,
            b_prior,
            sigma2_prior,
        )

        # Acceptance probability
        acceptance_prob = np.exp(proposed_log_post - current_log_post)
        # Accept or reject
        if np.random.rand() < acceptance_prob:
            m_current, b_current, sigma2_current = (
                m_proposal,
                b_proposal,
                sigma2_proposal,
            )
            current_log_post = proposed_log_post
            accepted += 1

        # Store samples after burn-in
        if iteration >= burn_in:
            m_samples.append(m_current)
            b_samples.append(b_current)
            sigma2_samples.append(sigma2_current)

    return m_samples, b_samples, sigma2_samples, accepted / (n_iterations - burn_in)

In [11]:
x_data = np.random.normal(0, 1, 100)
y_data = 3.0 * x_data + 4.0 + np.random.normal(0, np.sqrt(2), 100)

# Define priors: (mean, variance) for normal, (alpha, beta) for inverse gamma
m_prior = (4, 3)
b_prior = (3, 3)
sigma2_prior = (0.3, 3)

# Run Metropolis-Hastings
m_samples, b_samples, sigma2_samples, afrac = metropolis_hastings(
    y_data,
    x_data,
    m_prior=m_prior,
    b_prior=b_prior,
    sigma2_prior=sigma2_prior,
    proposal_std=(0.1, 0.1, 0.1),
    n_iterations=10000,
)
print(f"acceptance frac: {afrac}")
print("Estimated m:", np.mean(m_samples))
print("Estimated b:", np.mean(b_samples))
print("Estimated sigma^2:", np.mean(sigma2_samples))

acceptance frac: 0.5863265306122449
Estimated m: 2.96504916042215
Estimated b: 3.967211455661518
Estimated sigma^2: 1.996042291709049


## Alternate sampling error and model parameters in batches
This is useful when evaluating the model is compuationall expensive relative to evaluting the log likelihood conditional on a given model prediction

In [12]:
def log_posterior_batch(y, y_model, sigma2, m_prior, b_prior, sigma2_prior):
    """
    Calculate the log posterior given parameters, considering fixed y_model.
    """
    # Log likelihood
    residuals = y - y_model
    log_likelihood = -0.5 * np.sum(residuals**2 / sigma2) - (len(y) / 2) * np.log(
        2 * np.pi * sigma2
    )

    # Assuming m_prior and b_prior are already incorporated during m and b updates
    # Log prior for sigma^2
    log_prior_sigma2 = invgamma.logpdf(sigma2, a=sigma2_prior[0], scale=sigma2_prior[1])

    return log_likelihood + log_prior_sigma2

In [13]:
def metropolis_hastings_batched(
    y,
    x,
    n_iterations=1000,
    burn_in=200,
    batch_size=10,
    m_prior=(0, 10),
    b_prior=(0, 10),
    sigma2_prior=(1, 1),
    proposal_std=(0.1, 0.1, 0.1),
):
    prop_m, prop_b, prop_log_sigma2 = proposal_std

    # Initialization
    m_current = np.random.normal(m_prior[0], np.sqrt(m_prior[1]))
    b_current = np.random.normal(b_prior[0], np.sqrt(b_prior[1]))
    sigma2_current = invgamma.rvs(a=sigma2_prior[0], scale=sigma2_prior[1])
    y_model = m_current * x + b_current

    # Storage for samples
    m_samples = []
    b_samples = []
    sigma2_samples = []
    accepted = 0
    for iteration in range(n_iterations):
        if iteration % (2 * batch_size) < batch_size:
            # Update m and b
            m_proposal = m_current + np.random.normal(0, prop_m)
            b_proposal = b_current + np.random.normal(0, prop_b)
            y_model_proposal = m_proposal * x + b_proposal

            # Calculate and compare log posterior
            current_log_post = log_posterior_batch(
                y, y_model, sigma2_current, m_prior, b_prior, sigma2_prior
            )
            proposed_log_post = log_posterior_batch(
                y, y_model_proposal, sigma2_current, m_prior, b_prior, sigma2_prior
            )

            acceptance_prob = np.exp(proposed_log_post - current_log_post)
            if np.random.rand() < acceptance_prob:
                m_current, b_current, y_model = m_proposal, b_proposal, y_model_proposal
                accepted += 1

        else:
            # Update sigma^2
            log_sigma2_proposal = np.log(sigma2_current) + np.random.normal(
                0, prop_log_sigma2
            )
            sigma2_proposal = np.exp(log_sigma2_proposal)

            # Calculate and compare log posterior (holding m and b fixed)
            current_log_post = log_posterior_batch(
                y, y_model, sigma2_current, m_prior, b_prior, sigma2_prior
            )
            proposed_log_post = log_posterior_batch(
                y, y_model, sigma2_proposal, m_prior, b_prior, sigma2_prior
            )

            acceptance_prob = np.exp(proposed_log_post - current_log_post)
            if np.random.rand() < acceptance_prob:
                sigma2_current = sigma2_proposal
                accepted += 1

        # Store samples after burn-in
        if iteration >= burn_in:
            m_samples.append(m_current)
            b_samples.append(b_current)
            sigma2_samples.append(sigma2_current)

    return m_samples, b_samples, sigma2_samples, accepted / (n_iterations - burn_in)

In [14]:
x_data = np.random.normal(0, 1, 100)
y_data = 3.0 * x_data + 4.0 + np.random.normal(0, np.sqrt(0.8), 100)

# Define priors: (mean, variance) for normal, (alpha, beta) for inverse gamma
reported_stat_err = 0.3
m_prior = (2, 10)
b_prior = (1, 10)
sigma2_prior = (3, reported_stat_err / (3 - 1))

# Run Metropolis-Hastings in batches
m_samples, b_samples, sigma2_samples, acc_frac = metropolis_hastings_batched(
    y_data,
    x_data,
    m_prior=m_prior,
    b_prior=b_prior,
    sigma2_prior=sigma2_prior,
    proposal_std=(0.1, 0.1, 0.1),
    n_iterations=10000,
)
print(f"Acceptance fraction {acc_frac}")
print("Estimated m:", np.mean(m_samples))
print("Estimated b:", np.mean(b_samples))
print("Estimated sigma^2:", np.mean(sigma2_samples))

/tmp/ipykernel_65725/530061145.py:40: RuntimeWarning: overflow encountered in exp
  acceptance_prob = np.exp(proposed_log_post - current_log_post)
/tmp/ipykernel_65725/530061145.py:60: RuntimeWarning: overflow encountered in exp
  acceptance_prob = np.exp(proposed_log_post - current_log_post)


Acceptance fraction 0.6506122448979592
Estimated m: 2.970686404535775
Estimated b: 4.125086047917617
Estimated sigma^2: 0.6888285329155968
